In [1]:
from FlagEmbedding import FlagModel
import numpy as np
from sklearn.manifold import TSNE
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from pprint import pprint as pp
import time
import umap
import os
import random
import time
from contextlib import contextmanager
import torch
from sentence_transformers import SentenceTransformer
from datasets import load_dataset, load_from_disk
from datasets import load_dataset, concatenate_datasets, load_from_disk
import pandas as pd
import datasets
from datasets import Dataset
from pprint import pprint as pp
from datasets import Dataset
from sklearn.cluster import KMeans
from tqdm import tqdm
import torch
import heapq

In [2]:
from datasets import load_dataset, load_from_disk
from datasets import load_dataset, concatenate_datasets, load_from_disk
import pandas as pd
import datasets
from datasets import Dataset
from pprint import pprint as pp
from datasets import Dataset
from sklearn.cluster import KMeans
from tqdm import tqdm
import torch
import heapq
code_data = load_dataset("sahil2801/CodeAlpaca-20k")["train"]
fin_data = load_dataset("FinGPT/fingpt-sentiment-train")["train"]
med_data = load_dataset("medalpaca/medical_meadow_medical_flashcards")["train"]
general_data = load_dataset("tatsu-lab/alpaca")["train"]
math_data = load_dataset("TIGER-Lab/MathInstruct")["train"]

def alpaca_format(example):
    if example['input'] == "":
        example["instruction"] = example["instruction"]
    else:
        example["instruction"] = example["instruction"] + " " + example['input']
    example["response"] = example['output']
    return example

def process_sft_dataset(dataset_name, dataset, dataset_sample=None)->datasets.Dataset:
    if dataset_name in ["lucasmccabe-lmi/CodeAlpaca-20k", "yahma/alpaca-cleaned", "FinGPT/fingpt-sentiment-train"]:
        dataset = dataset.map(alpaca_format, remove_columns=['input', 'output'], desc=f"Preprocessing {dataset_name} for unified format.")
    elif dataset_name in ["WizardLM/WizardLM_evol_instruct_70k"]:
        dataset = dataset.rename_column("output", "response")
    elif dataset_name in ["tatsu-lab/alpaca", "vicgalle/alpaca-gpt4", "gbharti/finance-alpaca"]:
        dataset = dataset.map(alpaca_format, remove_columns=['input', 'output', 'text'], desc=f"Preprocessing {dataset_name} for unified format.")
    elif dataset_name in ["TIGER-Lab/MathInstruct"]:
        df = pd.DataFrame(dataset)
        df = df.drop_duplicates(subset=['instruction'])
        dataset = datasets.Dataset.from_pandas(df)
        # dataset = dataset.shuffle(seed=42).select(range(51000))
        dataset = dataset.rename_column("output", "response")
        dataset = dataset.remove_columns(['source'])
    elif dataset_name in ["lighteval/MATH"]:
        dataset = dataset.rename_column("solution", "response")
        dataset = dataset.rename_column("problem", "instruction")
        dataset = dataset.remove_columns(['level', 'type'])
    elif dataset_name in ['gsm8k']:
        dataset = dataset.rename_column("question", "instruction")
        dataset = dataset.rename_column("answer", "response")
    elif dataset_name in ['medalpaca/medical_meadow_medical_flashcards']:       # TODO: 'lavita/ChatDoctor-HealthCareMagic-100k'. not sure whether to discard the instruction.
        dataset = dataset.remove_columns(['instruction'])
        dataset = dataset.rename_column("input", "instruction")
        dataset = dataset.rename_column("output", "response")
    elif "math" in dataset_name:
        dataset = dataset.remove_columns(['source'])
        dataset = dataset.rename_column("output", "response")
    else:
        raise NotImplementedError(f"Dataset {dataset_name} is not supported.")
    dataset = dataset.shuffle(seed=42)
    if dataset_sample:
        num_sample = min(len(dataset), dataset_sample)
        dataset = dataset.select(range(num_sample))
    print(f">> ===== After processing, Dataset {dataset_name} has {len(dataset)} examples. =====")
    return dataset

In [3]:
processed_data = []
# 这块一定要注意!!! name 和datasest的顺序都要改
for name, dataset in zip(["medalpaca/medical_meadow_medical_flashcards","lucasmccabe-lmi/CodeAlpaca-20k","TIGER-Lab/MathInstruct","FinGPT/fingpt-sentiment-train","tatsu-lab/alpaca",],[med_data,code_data,math_data,fin_data,general_data]):
# for name, dataset in zip(["lucasmccabe-lmi/CodeAlpaca-20k","FinGPT/fingpt-sentiment-train","medalpaca/medical_meadow_medical_flashcards", "TIGER-Lab/MathInstruct"],[code_data,fin_data,med_data,math_data]):
    tmp:datasets.Dataset = process_sft_dataset(name,dataset)
    # if "fin" in name: 
    #     tmp = tmp.shuffle(seed=42).select(range(51000))
    print(tmp.column_names)
    processed_data.append(tmp)

>> ===== After processing, Dataset medalpaca/medical_meadow_medical_flashcards has 33955 examples. =====
['instruction', 'response']
>> ===== After processing, Dataset lucasmccabe-lmi/CodeAlpaca-20k has 20022 examples. =====
['instruction', 'response']


>> ===== After processing, Dataset TIGER-Lab/MathInstruct has 224567 examples. =====
['response', 'instruction', '__index_level_0__']
>> ===== After processing, Dataset FinGPT/fingpt-sentiment-train has 76772 examples. =====
['instruction', 'response']
>> ===== After processing, Dataset tatsu-lab/alpaca has 52002 examples. =====
['instruction', 'response']


In [9]:
import openai
from tqdm import tqdm

from openai import OpenAI

client = OpenAI(api_key="", base_url="https://api.deepseek.com")

instructions = processed_data[0]["instruction"]
responses = processed_data[0]["response"]

# 定义判断主题的提示信息
def check_topic(instruction):
    prompt = f"""
    你是一名医学专家。请判断以下医学指令是否属于以下主题之一，并给出理由：
    - clinical knowledge
    - anatomy
    - medical genetics

    ## Requirement: 如果属于以上任何一个主题，请回答“属于”；否则，请回答“不属于”.

    ## Input: "{instruction}"
    ## Output:
    """
    response = client.chat.completions.create(
        # model="gpt-4o-2024-08-06",  # 使用GPT-4模型
        model="deepseek-chat",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=256,
        temperature=0.5,  # 确保答案稳定
        n=1,
        # stop=["\n"],
    )
    # 提取回答结果
    answer = response.choices[0].message.content.strip()
    print(answer)
    return answer == "属于"

# 分配两个列表：包含指定主题的和不包含指定主题的
included_topics = []
excluded_topics = []

# 对每条数据进行主题判断并分类
for instruction, response in tqdm(zip(instructions, responses), total=len(instructions)):
    # print(instruction)
    if check_topic(instruction):
        included_topics.append({"instruction": instruction, "response": response})
    else:
        excluded_topics.append({"instruction": instruction, "response": response})
    break

# 输出结果
# print("包含指定主题的指令集：", len(included_topics))
# print("不包含指定主题的指令集：", len(excluded_topics))

# # 返回结果以便于进一步使用
# print(included_topics[0])
# print(excluded_topics[0])

  0%|          | 0/33955 [00:00<?, ?it/s]

  0%|          | 0/33955 [00:04<?, ?it/s]

属于

理由：这个问题涉及到上肢（arm/elbow）的损伤类型，特别是导致肱骨髁上骨折（supracondylar fractures）的常见原因。这属于解剖学（anatomy）和临床知识（clinical knowledge）的范畴，因为它涉及到特定部位的解剖结构和常见的临床损伤模式。


In [6]:
instructions = processed_data[0]["instruction"]
responses = processed_data[0]["response"]

In [7]:
system_prompt = """You are a medical professional. Please determine whether the following medical instructions fall under one of the following topics:
- clinical knowledge
- anatomy
- medical genetics"""

instruction = f"{instructions[0]}"

user_prompt = f"""
Inpute: "{instruction}"
If you belong to any of the above topics, please answer "Yes"; otherwise, please answer "No".
"""

prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>

{user_prompt}<|eot_id|>"""

print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a medical professional. Please determine whether the following medical instructions fall under one of the following topics:
- clinical knowledge
- anatomy
- medical genetics<|eot_id|><|start_header_id|>user<|end_header_id|>


Inpute: "What type of injury to the arm/elbow most often leads to supracondylar fractures?"
If you belong to any of the above topics, please answer "Yes"; otherwise, please answer "No".
<|eot_id|>


In [8]:
prompts = [prompt.format_map({"instruction": ins}) for ins in instructions[:100]]

In [3]:
from vllm import LLM, SamplingParams
llm = LLM(model="/mnt/bn/merlin-datavolume-tsy/leon/checkpoints/Llama3.1-8B-Chinese-Chat", tensor_parallel_size=1, 
            dtype=torch.bfloat16, trust_remote_code=True, enable_lora=False, max_model_len=2048, gpu_memory_utilization=0.8)

sampling_params = SamplingParams(top_p=1, max_tokens=10, temperature=0) # memory extraction

INFO 11-14 11:09:11 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='/mnt/bn/merlin-datavolume-tsy/leon/checkpoints/Llama3.1-8B-Chinese-Chat', speculative_config=None, tokenizer='/mnt/bn/merlin-datavolume-tsy/leon/checkpoints/Llama3.1-8B-Chinese-Chat', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=/mnt/bn/merlin-datavolume

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 11-14 11:09:16 model_runner.py:1067] Loading model weights took 14.9888 GB
INFO 11-14 11:09:17 gpu_executor.py:122] # GPU blocks: 20011, # CPU blocks: 2048
INFO 11-14 11:09:17 gpu_executor.py:126] Maximum concurrency for 2048 tokens per request: 156.34x
INFO 11-14 11:09:18 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-14 11:09:18 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-14 11:09:34 model_runner.py:1523] Graph capturing finished in 16 secs.


In [14]:
outputs = llm.generate("介绍一下 python", sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  7.26it/s, est. speed input: 21.93 toks/s, output: 73.08 toks/s]


In [15]:
print(outputs[0].outputs[0].text)

 的内置函数 `enumerate` 的用法


In [10]:
for output in outputs:
    print(output.outputs[0].text)

<|start_header_id|>assistant<|end_header_id|>

Yes, this question falls under
<|start_header_id|>assistant<|end_header_id|>

Yes, this question falls under
<|start_header_id|>assistant<|end_header_id|>

Yes, this question falls under
<|start_header_id|>assistant<|end_header_id|>

Yes, this question falls under
<|start_header_id|>assistant<|end_header_id|>

Yes, this question falls under
<|start_header_id|>assistant<|end_header_id|>

Yes, this question falls under
<|start_header_id|>assistant<|end_header_id|>

Yes, this question falls under
<|start_header_id|>assistant<|end_header_id|>

Yes, this question falls under
<|start_header_id|>assistant<|end_header_id|>

Yes, this question falls under
<|start_header_id|>assistant<|end_header_id|>

Yes, this question falls under
<|start_header_id|>assistant<|end_header_id|>

Yes, this question falls under
<|start_header_id|>assistant<|end_header_id|>

Yes, this question falls under
<|start_header_id|>assistant<|end_header_id|>

Yes, this questio